# Load Data

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd

# Load the data
demographics = pd.read_csv('https://storage.googleapis.com/datalynn-datasets/Interview_Challenge/Amex/Amex_Marketing_Strategies/demographics.csv')
transactions = pd.read_csv('https://storage.googleapis.com/datalynn-datasets/Interview_Challenge/Amex/Amex_Marketing_Strategies/transactions.csv')
campaigns = pd.read_csv('https://storage.googleapis.com/datalynn-datasets/Interview_Challenge/Amex/Amex_Marketing_Strategies/campaigns.csv')
transactions['TransactionDate'] = pd.to_datetime(transactions['TransactionDate'])
campaigns['CampaignDate'] = pd.to_datetime(campaigns['CampaignDate'])

# Merge the data
data = pd.merge(transactions, demographics, on='CustomerID')
data = pd.merge(data, campaigns, on='CustomerID')

# Let's have a quick look at the data
print(data.head())

   TransactionID  CustomerID  TransactionAmount TransactionDate  \
0              1           1         130.221277      2022-01-01   
1              1           1         130.221277      2022-01-01   
2              1           1         130.221277      2022-01-01   
3              1           1         130.221277      2022-01-01   
4              1           1         130.221277      2022-01-01   

  MerchantCategory  Age  Income Occupation  Gender  CampaignID CampaignDate  \
0           Dining   38   75034     Doctor  Female           1   2022-01-01   
1           Dining   38   75034     Doctor  Female           2   2022-01-02   
2           Dining   38   75034     Doctor  Female           3   2022-01-03   
3           Dining   38   75034     Doctor  Female           4   2022-01-04   
4           Dining   38   75034     Doctor  Female           5   2022-01-05   

   CampaignResponse  
0                 0  
1                 1  
2                 1  
3                 1  
4           

# Data Manipulation
**Q1** As you can see, our dataset contains information on different merchant categories. However, the categorization is inconsistent across different entries. How would you standardize these categories?

In [ ]:
# Standardize merchant categories
data['MerchantCategory'] = data['MerchantCategory'].str.lower().str.strip()

# Print the unique categories to verify
print(data['MerchantCategory'].unique())



['dining' 'clothing' 'entertainment' 'grocery' 'electronics']


**Q2** For certain transactions, you might notice some anomalies such as negative transaction amounts or transactions that take place in the future. Can you devise a strategy to identify and handle these outliers?

In [ ]:
# Identify negative transaction amounts
negative_transactions = data[data['TransactionAmount'] < 0]

# Print negative transactions
print(negative_transactions)

# Identify transactions in the future
future_transactions = data[data['TransactionDate'] > pd.Timestamp.today()]

# Print future transactions
print(future_transactions)



Empty DataFrame
Columns: [TransactionID, CustomerID, TransactionAmount, TransactionDate, MerchantCategory, Age, Income, Occupation, Gender, CampaignID, CampaignDate, CampaignResponse]
Index: []
       TransactionID  CustomerID  TransactionAmount TransactionDate  \
2674             546          58          93.652764      2023-06-30   
2675             546          58          93.652764      2023-06-30   
2676             546          58          93.652764      2023-06-30   
2677             546          58          93.652764      2023-06-30   
2678             546          58          93.652764      2023-06-30   
...              ...         ...                ...             ...   
43418           8948        1000         129.415168      2046-07-01   
43419           8948        1000         129.415168      2046-07-01   
43420           8948        1000         129.415168      2046-07-01   
43421           8948        1000         129.415168      2046-07-01   
43422           8948     

In [ ]:
# Remove anomalies
data = data[data['TransactionAmount'] >= 0]
data = data[data['TransactionDate'] <= pd.Timestamp.today()]

# Verify the removal
print(data[data['TransactionAmount'] < 0])
print(data[data['TransactionDate'] > pd.Timestamp.today()])


Empty DataFrame
Columns: [TransactionID, CustomerID, TransactionAmount, TransactionDate, MerchantCategory, Age, Income, Occupation, Gender, CampaignID, CampaignDate, CampaignResponse]
Index: []
Empty DataFrame
Columns: [TransactionID, CustomerID, TransactionAmount, TransactionDate, MerchantCategory, Age, Income, Occupation, Gender, CampaignID, CampaignDate, CampaignResponse]
Index: []


# Feature Engineering
**Q3** Using this combined dataset, how would you create a feature that encapsulates the change in a customer's spending habits after a successful marketing campaign interaction?


In [ ]:
# Convert dates to datetime format
data['TransactionDate'] = pd.to_datetime(data['TransactionDate'])
data['CampaignDate'] = pd.to_datetime(data['CampaignDate'])

# Calculate pre and post campaign average transaction amounts for each campaign
pre_campaign_avg = data.loc[data['TransactionDate'] < data['CampaignDate']].groupby(['CustomerID', 'CampaignID'])['TransactionAmount'].mean()
post_campaign_avg = data.loc[data['TransactionDate'] > data['CampaignDate']].groupby(['CustomerID', 'CampaignID'])['TransactionAmount'].mean()

# Compute difference
spending_change = post_campaign_avg - pre_campaign_avg

# Merge back to original data
data = pd.merge(data, spending_change.rename('SpendingChange'), how='left', on=['CustomerID', 'CampaignID'])



**Q4** Can you devise a way to quantify a customer's responsiveness to our marketing campaigns? This might consider not just whether the customer responded, but how quickly they responded and how their behavior changed post-campaign.


In [ ]:
# Calculate time to next transaction after each campaign
data['TimeToNextTransaction'] = data.sort_values('TransactionDate').groupby(['CustomerID', 'CampaignID'])['TransactionDate'].diff().dt.days
# Some customers might not have a transaction after a campaign, fill NA values with a large number
data['TimeToNextTransaction'].fillna(999, inplace=True)
df = data.sort_values('TransactionDate').groupby(['CustomerID', 'CampaignID'])['TransactionDate']


**Q5** How would you engineer a feature that represents the trend in a customer's income? Consider how you might capture changes in income over time, given that income might be related to a customer's transaction behavior.

In [ ]:
# Compute trend in transaction amounts
data['TransactionTrend'] = data.sort_values('TransactionDate').groupby('CustomerID')['TransactionAmount'].pct_change()



# Modeling Metric & Model Selection

**Q6** When evaluating a model's performance, why might accuracy not be the best metric here? What other metrics would you consider and why?


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# Load the data
# We'll assume that data is a pandas DataFrame loaded with your data
# And that "CampaignSuccess" is the target variable

# Define the feature matrix X and the target y
X = data[['TransactionAmount', 'Age', 'Income', 'Occupation', 'Gender',  'TimeToNextTransaction']]
X = pd.get_dummies(X, columns=['Occupation', 'Gender'])
y = data['CampaignResponse']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = LogisticRegression()

# Fit the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# Print the metrics
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"ROC AUC: {roc_auc}")



Precision: 0.5555555555555556
Recall: 0.7386363636363636
F1 Score: 0.6341463414634146
ROC AUC: 0.5738636363636364


**Q7** Explain how you would handle class imbalance in this prediction problem. How does class imbalance impact the choice of models and evaluation metrics?


In [ ]:
from sklearn.linear_model import LogisticRegression

# Assume X is the feature matrix and y are the labels
# X = ...
# y = ...

model = LogisticRegression(class_weight='balanced')
# Fit the model
model.fit(X_train, y_train)

# Get predictions
y_pred = model.predict(X_test)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# Print the metrics
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"ROC AUC: {roc_auc}")


Precision: 0.563573883161512
Recall: 0.6212121212121212
F1 Score: 0.590990990990991
ROC AUC: 0.5700757575757576


**Q8** How would you approach the problem if the model's performance is significantly different on the validation set than on the training set? What steps would you take to ensure your model is not overfitting?

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X, y, cv=5, scoring='roc_auc')

print(f"Cross-validated ROC AUC: {np.mean(scores)}")


Cross-validated ROC AUC: 0.587368567675048


# Business Insight
**Q9** After applying your model to the unified dataset, can you determine if there are any demographic groups that our campaigns are particularly effective or ineffective with? How might this information be useful for optimizing future campaigns?



In [ ]:
# Group data by occupation and calculate average response
occupation_effectiveness = data.groupby('Occupation')['CampaignResponse'].mean()

# Print the results
print(occupation_effectiveness)


Occupation
Artist            0.343008
Data Scientist    0.728997
Doctor            0.563177
Engineer          0.462299
Teacher           0.538188
Name: CampaignResponse, dtype: float64


**Q10** If a competitor has successfully increased their customer engagement with a similar marketing strategy, how would you incorporate this information into your analysis to further optimize our own strategy?


**Q11** How would you estimate the potential increase in revenue that could result from implementing your recommendations?


In [ ]:
# Calculate the existing average transaction amount and response rate
avg_transaction_amount = data['TransactionAmount'].mean()
avg_response_rate = data['CampaignResponse'].mean()

# Assume an estimated increase in response rate based on the recommendations
estimated_increase = 0.05 # This is just an example

# Calculate the potential increase in revenue
potential_revenue_increase = avg_transaction_amount * estimated_increase * len(data)

print(potential_revenue_increase)

13326.64553183677


**Q12** What are the potential risks of implementing your recommendations and how would you mitigate these risks? How would you track and measure the success of implementing your recommendations?